In [2]:
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import TensorBoard
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

MODEL_NAME = "MegaClassifier_a"
VERSION_BASE = "v7"
EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset['file_name'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
dataset['binary_label'] = dataset['binary_label'].astype(str)

train_df = dataset[dataset['subset'] == 'train']
validation_df = dataset[dataset['subset'] == 'validation']
test_df = dataset[dataset['subset'] == 'test']

class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.unique(train_df['binary_label']),
                                     y=train_df['binary_label'])
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


def focal_loss(alpha=0.25, gamma=1.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
        return loss

    return loss


brightness_low_values = [0.7, 0.8, 0.9, 1.0]
brightness_high_values = [1.1, 1.2, 1.3, 1.4]


def train_and_evaluate(brightness_low, brightness_high, VERSION_INDEX):
    brightness_label = f"Brightness_{brightness_low}-{brightness_high}"
    print(f"\n🔹 Entrenando con Brightness {brightness_label} 🔹")

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        brightness_range=[brightness_low, brightness_high],
        horizontal_flip=True
    )

    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    validation_images = datagen.flow_from_dataframe(
        dataframe=validation_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    test_images = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
    )

    mobilenet_v2 = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    mobilenet_v2.trainable = False

    model = tf.keras.Sequential([
        mobilenet_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ], name=f"{MODEL_NAME}_{VERSION_BASE}_{VERSION_INDEX}")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss=focal_loss(alpha=0.25, gamma=1.0),
        metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")],
    )

    start_time = time.time()
    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        class_weight=class_weight_dict,
        callbacks=[
            TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}"),
        ]
    )
    training_time = time.time() - start_time

    results = model.evaluate(test_images)

    history_df = pd.DataFrame(history.history)
    os.makedirs(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}", exist_ok=True)
    history_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/history_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    metric_names = history.model.metrics_names
    evaluation_results = {("test_" + name): value for name, value in zip(metric_names, results)}
    evaluation_results["brightness_range"] = f"{brightness_low}-{brightness_high}"
    evaluation_results["training_time"] = training_time

    results_df = pd.DataFrame([evaluation_results])
    results_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/results_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    print(f"\n📉 Loss: {results[0]:.4f}")
    print(f"🎯 Accuracy: {results[1]:.4%}")
    print(f"✅ Precision: {results[2]:.4%}")
    print(f"🔄 Recall: {results[3]:.4%}")
    print(f"📊 AUC: {results[4]:.4f}")
    print(f"⏳ Tiempo de entrenamiento: {training_time:.2f} segundos")

    return results_df


count = 0
all_results = []
for brightness_low in brightness_low_values:
    for brightness_high in brightness_high_values:
        if brightness_low < brightness_high:  # Asegurar rangos válidos
            results_df = train_and_evaluate(brightness_low, brightness_high, count)
            all_results.append(results_df)
            count += 1

final_results = pd.concat(all_results, ignore_index=True)
final_results.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/brightness_optimization_results.csv", index=False)

print("\n✅ ¡Evaluación de Brightness completada!")


🔹 Entrenando con Brightness Brightness_0.7-1.1 🔹
Found 17054 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


Epoch 1/10


2025-02-24 11:48:00.122228: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0352 - accuracy: 0.8838 - precision: 0.9115 - recall: 0.9416 - auc: 0.9305

2025-02-24 11:48:45.595318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

Epoch 1/10


2025-02-24 15:13:37.891637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0395 - accuracy: 0.8645 - precision: 0.8949 - recall: 0.9350 - auc: 0.9113

2025-02-24 15:14:23.110004: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - 57s 105ms/step - loss: 0.0395 - accuracy: 0.8645 - precision: 0.8949 - recall: 0.9350 - auc: 0.9113 - val_loss: 0.0262 - val_accuracy: 0.9202 - val_precision: 0.9345 - val_recall: 0.9457 - val_auc: 0.9724
Epoch 2/10
533/533 [==============================] - 55s 103ms/step - loss: 0.0269 - accuracy: 0.9110 - precision: 0.9376 - recall: 0.9483 - auc: 0.9611 - val_loss: 0.0234 - val_accuracy: 0.9270 - val_precision: 0.9331 - val_recall: 0.9584 - val_auc: 0.9781
Epoch 3/10
533/533 [==============================] - 54s 100ms/step - loss: 0.0247 - accuracy: 0.9187 - precision: 0.9450 - recall: 0.9504 - auc: 0.9674 - val_loss: 0.0222 - val_accuracy: 0.9326 - val_precision: 0.9405 - val_recall: 0.9587 - val_auc: 0.9804
Epoch 4/10
533/533 [==============================] - 54s 101ms/step - loss: 0.0234 - accuracy: 0.9236 - precision: 0.9489 - recall: 0.9527 - auc: 0.9710 - val_loss: 0.0216 - val_accuracy: 0.9365 - val_precision: 0.9439 - val_recall: 